In [103]:
import pandas as pd
import numpy as np
from scipy.stats import zscore, mstats

# Let's try to replicate the MSCI Momentum Index methodology

- Uses both 6-month and 12-month returns
- Final score winsorized at +/-3
- Final weight capped at 7%

In [104]:
# Load market data
market_data = pd.read_csv('market_data.csv')

# Convert date column to datetime
market_data['date'] = pd.to_datetime(market_data['date'])

# Extract year and month from date
market_data['year_month'] = market_data['date'].dt.to_period('M')

# Sort data by Ticker and date
market_data = market_data.sort_values(by=['Ticker', 'date'])

# Calculate monthly closing prices
monthly_prices = market_data.groupby(['Ticker', 'year_month']).agg({'Price': 'last'}).reset_index()

# Create a pivot table with Tickers as rows and year_month as columns
pivot_prices = monthly_prices.pivot(index='Ticker', columns='year_month', values='Price')

In [105]:
pivot_prices

year_month,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07
Ticker,,,,,,,,,,,,,
A-USA,121.77,121.07,111.82,103.37,127.80,139.03,130.10,137.36,145.51,137.04,130.41,129.63,139.49
AAL-USA,16.75,14.73,12.81,11.15,12.43,13.74,14.23,15.68,15.35,13.51,11.50,11.33,10.76
AAPL-USA,196.45,187.87,171.21,170.77,189.95,192.53,184.40,180.75,171.48,170.33,192.25,210.62,218.80
ABBV-USA,149.58,146.96,149.06,141.18,142.39,154.97,164.40,176.05,182.10,162.64,161.24,171.52,186.78
ABNB-USA,152.19,131.55,137.21,118.29,126.34,136.14,144.14,157.47,164.96,158.57,144.93,151.63,138.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL-USA,112.75,103.54,91.03,93.54,105.13,114.36,112.44,127.05,129.24,130.70,141.02,135.63,133.48
YUM-USA,137.67,129.38,124.94,120.86,125.55,130.66,129.49,138.42,138.65,141.25,137.43,132.46,132.17
ZBH-USA,138.15,119.12,112.22,104.41,116.31,121.70,125.60,124.36,131.98,120.28,115.15,108.53,111.37


In [106]:
# Calculate 6-month price change excluding the most recent month
momentum_value_6m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -7]) - 1

# Create DataFrame for 6-month momentum
momentum_df_6m = pd.DataFrame(momentum_value_6m, columns=['Momentum Value 6M'])

# Calculate 12-month price change excluding the most recent month
momentum_value_12m = (pivot_prices.iloc[:, -2] / pivot_prices.iloc[:, -13]) - 1

# Calculate the 12-month price change excluding the most recent month (already calculated as momentum_value)
momentum_df_12m = pd.DataFrame(momentum_value_12m, columns=['Momentum Value 12M'])

# Merge 6-month and 12-month momentum DataFrames
momentum_df = momentum_df_6m.join(momentum_df_12m)

# Filter out rows where either 6M or 12M momentum is zero
momentum_df = momentum_df[(momentum_df['Momentum Value 6M'] != 0) & (momentum_df['Momentum Value 12M'] != 0)]

# Calculate weighted average momentum
momentum_df['Weighted Momentum'] = (momentum_df['Momentum Value 6M'] + momentum_df['Momentum Value 12M']) / 2

# Merge market cap data
latest_market_data = market_data.sort_values('date').drop_duplicates('Ticker', keep='last')
momentum_df = momentum_df.merge(latest_market_data[['Ticker', 'Market Cap (millions)']], left_index=True, right_on='Ticker')

In [107]:
# Save the df here for the S&P Momentum Index analysis later in the notebook
SP_momentum_df = momentum_df

In [108]:
momentum_df = momentum_df[['Ticker', 'Market Cap (millions)', 'Momentum Value 6M', 'Momentum Value 12M', 'Weighted Momentum']].sort_values(by="Weighted Momentum",ascending=False)
momentum_df.dropna(subset=['Momentum Value 6M', 'Momentum Value 12M', 'Market Cap (millions)'], inplace=True)

# Standardize with z-scores
momentum_df['Final Z-score'] = zscore(momentum_df['Weighted Momentum'])
# Winsorize the final Z-scores (limit extreme values)
momentum_df['Clipped Z-score'] = momentum_df['Final Z-score'].clip(-3, 3)

# Limit to top 125
momentum_df = momentum_df.sort_values(by="Clipped Z-score",ascending=False).head(125)

# Remove '-USA' suffix from tickers in the momentum_df
momentum_df['Ticker'] = momentum_df['Ticker'].str.replace('-USA', '')

In [109]:
# Calculate weights based on Clipped Z-score and Market Cap
momentum_df['Weight_unnormalized'] = momentum_df['Clipped Z-score'] * momentum_df['Market Cap (millions)']
total_weight = momentum_df['Weight_unnormalized'].sum()
momentum_df['Final Weight'] = (momentum_df['Weight_unnormalized'] / total_weight) * 100

In [110]:
momentum_df = momentum_df.sort_values(by="Final Weight",ascending=False)
momentum_df

,Ticker,Market Cap (millions),Momentum Value 6M,Momentum Value 12M,Weighted Momentum,Final Z-score,Clipped Z-score,Weight_unnormalized,Final Weight
91175,NVDA,2.551758e+06,1.007899,1.643754,1.325827,5.865268,3.000000,7.655274e+06,25.078062
8645,AMZN,1.890988e+06,0.245168,0.445616,0.345392,1.280233,1.280233,2.420906e+06,7.930694
85673,MSFT,3.143271e+06,0.124176,0.330525,0.227351,0.728210,0.728210,2.288960e+06,7.498448
74931,LLY,7.510199e+05,0.402364,0.991816,0.697090,2.924960,2.924960,2.196703e+06,7.196223
80695,META,1.015056e+06,0.292408,0.582611,0.437510,1.711025,1.711025,1.736786e+06,5.689571
...,...,...,...,...,...,...,...,...,...
91961,NWSA,1.028310e+04,0.118912,0.391019,0.254966,0.857352,0.857352,8.816239e+03,0.028881
53447,GEN,1.613578e+04,0.063884,0.284319,0.174101,0.479188,0.479188,7.732069e+03,0.025330
84887,MRO,1.558994e+04,0.254705,0.091359,0.173032,0.474185,0.474185,7.392523e+03,0.024217
104537,RL,7.059164e+03,0.218487,0.332978,0.275732,0.954468,0.954468,6.737748e+03,0.022072


### Add caps to portfolio

MSCI caps the weight, though it's not totally clear what the methodology is to do that. They also have rules during reconstitution.
Let's cap the weight at 7%, which is around NVDA weight in SPMO, then ensure the rest of the portfolio is weighted accordingly.

In [111]:
# Strip trailing spaces from the 'Ticker' column in mtum_holdings_df
mtum_holdings_df['Ticker'] = mtum_holdings_df['Ticker'].str.strip()

# Step 1: Identify and cap weights greater than 7%
cap = 7
momentum_df['Capped Weight'] = momentum_df['Final Weight'].clip(upper=cap)

# Step 2: Calculate the total weight of capped and uncapped rows
total_capped_weight = momentum_df.loc[momentum_df['Final Weight'] > cap, 'Capped Weight'].sum()
total_uncapped_weight = momentum_df.loc[momentum_df['Final Weight'] <= cap, 'Final Weight'].sum()

# Step 3: Calculate the remaining total weight to be distributed
remaining_total_weight = 100 - total_capped_weight

# Step 4: Calculate the adjusted weights for uncapped rows
momentum_df['Adjusted Final Weight'] = momentum_df.apply(
    lambda row: row['Capped Weight'] if row['Final Weight'] > cap else 
                row['Final Weight'] / total_uncapped_weight * remaining_total_weight,
    axis=1
)

# Drop the temporary Capped Weight column
momentum_df.drop(columns=['Capped Weight'], inplace=True)

In [112]:
momentum_df = momentum_df.round(decimals=2)
momentum_df

,Ticker,Market Cap (millions),Momentum Value 6M,Momentum Value 12M,Weighted Momentum,Final Z-score,Clipped Z-score,Weight_unnormalized,Final Weight,Adjusted Final Weight
91175,NVDA,2551758.00,1.01,1.64,1.33,5.87,3.00,7655274.00,25.08,7.00
8645,AMZN,1890988.19,0.25,0.45,0.35,1.28,1.28,2420906.24,7.93,7.00
85673,MSFT,3143270.73,0.12,0.33,0.23,0.73,0.73,2288959.88,7.50,7.00
74931,LLY,751019.85,0.40,0.99,0.70,2.92,2.92,2196703.09,7.20,7.00
80695,META,1015056.01,0.29,0.58,0.44,1.71,1.71,1736785.92,5.69,7.83
...,...,...,...,...,...,...,...,...,...,...
91961,NWSA,10283.10,0.12,0.39,0.25,0.86,0.86,8816.24,0.03,0.04
53447,GEN,16135.78,0.06,0.28,0.17,0.48,0.48,7732.07,0.03,0.03
84887,MRO,15589.94,0.25,0.09,0.17,0.47,0.47,7392.52,0.02,0.03
104537,RL,7059.16,0.22,0.33,0.28,0.95,0.95,6737.75,0.02,0.03


In [113]:
momentum_df.to_csv("momentum_MSCI_proxy.csv")

### How closely does this match the MSCI portfolio?

In [114]:
# Load CSV data
momentum_proxy_df = pd.read_csv("momentum_MSCI_proxy.csv")
mtum_holdings_df = pd.read_csv("MTUM_holdings.csv")

# Strip trailing spaces from the 'Ticker' column in mtum_holdings_df
mtum_holdings_df['Ticker'] = mtum_holdings_df['Ticker'].str.strip()

# Part 1: Finding common tickers
momentum_tickers = set(momentum_proxy_df['Ticker'])
mtum_tickers = set(mtum_holdings_df['Ticker'])

common_tickers = momentum_tickers.intersection(mtum_tickers)
num_common_tickers = len(common_tickers)

print(f"Number of common tickers: {num_common_tickers}")
print(f"Common tickers: {common_tickers}")

Number of common tickers: 68
Common tickers: {'WM', 'SMCI', 'CRWD', 'MPC', 'RSG', 'AMZN', 'DECK', 'ALL', 'NRG', 'SYF', 'COST', 'PWR', 'CTAS', 'GD', 'NXPI', 'NVDA', 'AMD', 'HLT', 'CPRT', 'META', 'TT', 'HIG', 'UBER', 'BSX', 'NFLX', 'TDG', 'CEG', 'INTU', 'DPZ', 'AXP', 'GM', 'KKR', 'CMG', 'GRMN', 'LDOS', 'WFC', 'ETN', 'MU', 'URI', 'ANET', 'APH', 'VST', 'LLY', 'AXON', 'NTAP', 'WDC', 'MCK', 'PANW', 'IR', 'RCL', 'LRCX', 'DVA', 'JPM', 'FICO', 'WAB', 'AVGO', 'STX', 'GE', 'QCOM', 'HWM', 'CAT', 'ISRG', 'C', 'GDDY', 'AMAT', 'PHM', 'PGR', 'KLAC'}


In [115]:
# Extract and rename columns
momentum_proxy_df = momentum_proxy_df[['Ticker', 'Adjusted Final Weight']]
momentum_proxy_df = momentum_proxy_df.rename(columns={'Adjusted Final Weight': 'Proxy Weight'})

mtum_holdings_df = mtum_holdings_df[['Ticker', 'Weight (%)']]
mtum_holdings_df = mtum_holdings_df.rename(columns={'Weight (%)': 'MTUM Weight'})

# Merge the dataframes on 'Ticker', using outer join to include all tickers
combined_df = pd.merge(momentum_proxy_df, mtum_holdings_df, on='Ticker', how='outer')

# Fill NaN values with 0 for weights
combined_df['Proxy Weight'].fillna(0, inplace=True)
combined_df['MTUM Weight'].fillna(0, inplace=True)

/tmp/ipykernel_3164/2991764532.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Proxy Weight'].fillna(0, inplace=True)
/tmp/ipykernel_3164/2991764532.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [116]:
combined_df.sort_values(by="Proxy Weight", ascending=False).head(20)

,Ticker,Proxy Weight,MTUM Weight
108,META,7.83,4.31
16,AVGO,7.07,6.30
104,LLY,7.00,5.28
122,NVDA,7.00,6.73
10,AMZN,7.00,4.52
113,MSFT,7.00,0.00
79,GOOGL,5.57,0.00
78,GOOG,5.35,0.00
36,COST,2.26,3.14
182,WMT,2.09,0.00


# Let's try to recreate the S&P 500 Momentum Index

- Uses only 12-month returns
- Final score winsorized at +/-3
- Max weight of 12% (close to SPMO NVDA weight)
- 100 total stocks in portfolio

In [117]:
SP_momentum_df = SP_momentum_df[['Ticker', 'Market Cap (millions)', 'Momentum Value 12M', 'Weighted Momentum']].sort_values(by="Weighted Momentum",ascending=False)
SP_momentum_df.dropna(subset=['Momentum Value 12M', 'Market Cap (millions)'], inplace=True)

# Standardize with z-scores
SP_momentum_df['Final Z-score'] = zscore(SP_momentum_df['Weighted Momentum'])
# Winsorize the final Z-scores (limit extreme values)
SP_momentum_df['Clipped Z-score'] = SP_momentum_df['Final Z-score'].clip(-3, 3)

# Limit to top 125
SP_momentum_df = SP_momentum_df.sort_values(by="Clipped Z-score",ascending=False).head(100)

# Remove '-USA' suffix from tickers in the momentum_df
SP_momentum_df['Ticker'] = SP_momentum_df['Ticker'].str.replace('-USA', '')

In [118]:
# Calculate weights based on Clipped Z-score and Market Cap
SP_momentum_df['Weight_unnormalized'] = SP_momentum_df['Clipped Z-score'] * SP_momentum_df['Market Cap (millions)']
SP_total_weight = SP_momentum_df['Weight_unnormalized'].sum()
SP_momentum_df['Final Weight'] = (SP_momentum_df['Weight_unnormalized'] / SP_total_weight) * 100

SP_momentum_df = SP_momentum_df.sort_values(by="Final Weight",ascending=False)

In [119]:
cap = 12
finished_capping = False

while not finished_capping:
    # Cap weights above the threshold
    SP_momentum_df['Capped Weight'] = SP_momentum_df['Final Weight'].clip(upper=cap)

    # Calculate the total weight of capped and uncapped rows
    total_capped_weight = SP_momentum_df.loc[SP_momentum_df['Final Weight'] > cap, 'Capped Weight'].sum()
    total_uncapped_weight = SP_momentum_df.loc[SP_momentum_df['Final Weight'] <= cap, 'Final Weight'].sum()

    # Calculate the remaining total weight to be distributed
    remaining_total_weight = 100 - total_capped_weight

    # Calculate new weights for uncapped rows
    SP_momentum_df['Adjusted Final Weight'] = SP_momentum_df.apply(
        lambda row: (row['Capped Weight'] if row['Final Weight'] > cap else 
                    row['Final Weight'] / total_uncapped_weight * remaining_total_weight), 
        axis=1)

    # Check if any uncapped weights are pushed over the cap
    if (SP_momentum_df['Adjusted Final Weight'] > cap).any():
        SP_momentum_df['Final Weight'] = SP_momentum_df['Adjusted Final Weight']
    else:
        finished_capping = True

# Drop the temporary Capped Weight column
SP_momentum_df.drop(columns=['Capped Weight'], inplace=True)

# Round values to 2 decimal places
SP_momentum_df = SP_momentum_df.round(decimals=2)

In [120]:
SP_momentum_df = SP_momentum_df.round(decimals=2)
SP_momentum_df.to_csv("momentum_SP_proxy.csv")
SP_momentum_df

,Ticker,Market Cap (millions),Momentum Value 12M,Weighted Momentum,Final Z-score,Clipped Z-score,Weight_unnormalized,Final Weight,Adjusted Final Weight
91175,NVDA,2551758.00,1.64,1.33,5.87,3.00,7655274.00,26.01,12.00
8645,AMZN,1890988.19,0.45,0.35,1.28,1.28,2420906.24,8.23,9.79
85673,MSFT,3143270.73,0.33,0.23,0.73,0.73,2288959.88,7.78,9.25
74931,LLY,751019.85,0.99,0.70,2.92,2.92,2196703.09,7.46,8.88
80695,META,1015056.01,0.58,0.44,1.71,1.71,1736785.92,5.90,7.02
...,...,...,...,...,...,...,...,...,...
38251,DPZ,15008.73,0.30,0.26,0.86,0.86,12970.14,0.04,0.05
121043,UHS,12836.66,0.33,0.25,0.82,0.82,10566.00,0.04,0.04
91961,NWSA,10283.10,0.39,0.25,0.86,0.86,8816.24,0.03,0.04
104537,RL,7059.16,0.33,0.28,0.95,0.95,6737.75,0.02,0.03


### How close does this match the actual S&P Momentum Index?

In [123]:
# Load csv data
momentum_sp_proxy_df = pd.read_csv("momentum_SP_proxy.csv")
SPMO_holdings_df = pd.read_csv("SPMO holdings.csv")

# Strip trailing spaces from the 'Holding Ticker' column in mtum_df (if they exist)
SPMO_holdings_df['Ticker'] = SPMO_holdings_df['Ticker'].str.strip()

# Extract the ticker columns from both dataframes
momentum_tickers = set(momentum_sp_proxy_df['Ticker'])
SPMO_holdings_df = set(SPMO_holdings_df['Ticker'])

# Find the common tickers (intersection of both sets)
common_tickers = momentum_tickers.intersection(mtum_tickers)

# Calculate the number of common tickers
num_common_tickers = len(common_tickers)

# Print the result
print(f"Number of common tickers: {num_common_tickers}")
print(f"Common tickers: {common_tickers}")

Number of common tickers: 60
Common tickers: {'WM', 'SMCI', 'CRWD', 'RSG', 'AMZN', 'DECK', 'ALL', 'NRG', 'SYF', 'COST', 'PWR', 'CTAS', 'NXPI', 'NVDA', 'TT', 'HLT', 'META', 'HIG', 'UBER', 'BSX', 'NFLX', 'TDG', 'CEG', 'DPZ', 'AXP', 'GM', 'KKR', 'CMG', 'GRMN', 'LDOS', 'WFC', 'ETN', 'MU', 'URI', 'ANET', 'APH', 'VST', 'LLY', 'AXON', 'NTAP', 'WDC', 'MCK', 'IR', 'RCL', 'LRCX', 'DVA', 'JPM', 'FICO', 'WAB', 'AVGO', 'STX', 'GE', 'QCOM', 'HWM', 'ISRG', 'C', 'GDDY', 'AMAT', 'PGR', 'KLAC'}


In [129]:
# Load CSV data
momentum_sp_proxy_df = pd.read_csv("momentum_SP_proxy.csv")
SPMO_holdings_df = pd.read_csv("SPMO holdings.csv")

# Strip trailing spaces from the 'Ticker' column in mtum_holdings_df
SPMO_holdings_df['Ticker'] = SPMO_holdings_df['Ticker'].str.strip()

# Extract relevant columns and rename to match
momentum_sp_proxy_df = momentum_sp_proxy_df[['Ticker', 'Adjusted Final Weight']]
momentum_sp_proxy_df = momentum_sp_proxy_df.rename(columns={'Adjusted Final Weight': 'Proxy Weight'})

SPMO_holdings_df = SPMO_holdings_df[['Ticker', 'Weight']]
SPMO_holdings_df = SPMO_holdings_df.rename(columns={'Weight': 'SPMO Weight'})

# Merge the dataframes on 'Ticker', using outer join to include all tickers
combined_df = pd.merge(momentum_sp_proxy_df, SPMO_holdings_df, on='Ticker', how='outer')

# Fill NaN values with 0 for weights
combined_df['Proxy Weight'].fillna(0, inplace=True)
combined_df['SPMO Weight'].fillna(0, inplace=True)

/tmp/ipykernel_3164/2114525039.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  combined_df['Proxy Weight'].fillna(0, inplace=True)
/tmp/ipykernel_3164/2114525039.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [130]:
combined_df.sort_values(by="Proxy Weight",ascending=False).head(20)

,Ticker,Proxy Weight,SPMO Weight
100,NVDA,12.00,11.943
12,AMZN,9.79,7.450
93,MSFT,9.25,8.590
83,LLY,8.88,4.865
89,META,7.02,7.940
15,AVGO,6.34,5.256
60,GOOGL,4.99,0.000
59,GOOG,4.79,0.000
34,COST,2.02,2.158
151,WMT,1.88,1.031
